# Introduction to `spotPython` {#sec-spot}

Surrogate model based optimization methods are common approaches in simulation and optimization. SPOT was developed because there is a great need for sound statistical analysis of simulation and optimization algorithms. SPOT includes methods for tuning based on classical regression and analysis of variance techniques.
It presents tree-based models such as classification and regression trees and random forests as well as Bayesian optimization (Gaussian process models, also known as Kriging). Combinations of different meta-modeling approaches are possible. SPOT comes with a sophisticated surrogate model based optimization method, that can handle discrete and continuous inputs. Furthermore, any model implemented in `scikit-learn` can be used out-of-the-box as a surrogate in `spotPython`.

SPOT implements key techniques such as exploratory fitness landscape analysis and sensitivity analysis. It can be used to understand the performance of various algorithms, while simultaneously giving insights into their algorithmic behavior.

The `spot` loop consists of the following steps:

1. Init: Build initial design $X$
2. Evaluate initial design on real objective $f$: $y = f(X)$
3. Build surrogate: $S = S(X,y)$
4. Optimize on surrogate: $X_0 =  \text{optimize}(S)$
5. Evaluate on real objective: $y_0 = f(X_0)$
6. Impute (Infill) new points: $X = X \cup X_0$, $y = y \cup y_0$.
7. Goto 3.

Central Idea: Evaluation of the surrogate model `S` is much cheaper (or / and much faster) than running the real-world experiment $f$.
We start with a small example.

## Example: `Spot` and the Sphere Function


In [1]:
import numpy as np
from math import inf
from spotPython.fun.objectivefunctions import analytical
from spotPython.utils.init import fun_control_init, design_control_init
from spotPython.hyperparameters.values import set_control_key_value
from spotPython.spot import spot
import matplotlib.pyplot as plt

Seed set to 123


### The Objective Function: Sphere

The `spotPython` package provides several classes of objective functions. We will use an analytical objective function, i.e., a function that can be described by a (closed) formula:
$$
f(x) = x^2
$$


In [2]:
fun = analytical().fun_sphere

We can apply the function `fun` to input values and plot the result:


In [3]:
x = np.linspace(-1,1,100).reshape(-1,1)
y = fun(x)
plt.figure()
plt.plot(x, y, "k")
plt.show()

<Figure size 1650x1050 with 1 Axes>

### The `Spot` Method as an Optimization Algorithm Using a Surrogate Model

We initialize the `fun_control` dictionary. The `fun_control` dictionary contains the parameters for the objective function. The `fun_control` dictionary is passed to the `Spot` method.


In [4]:
fun_control=fun_control_init(lower = np.array([-1]),
                     upper = np.array([1]))
spot_0 = spot.Spot(fun=fun,
                   fun_control=fun_control)
spot_0.run()

Seed set to 123


spotPython tuning: 1.1986325668379847e-08 [#######---] 73.33% 


spotPython tuning: 1.1986325668379847e-08 [########--] 80.00% 


spotPython tuning: 1.1986325668379847e-08 [#########-] 86.67% 


spotPython tuning: 1.1385311249270152e-08 [#########-] 93.33% 


spotPython tuning: 5.189351944607845e-10 [##########] 100.00% Done...



Print the results:


In [5]:
spot_0.print_results()

min y: 5.189351944607845e-10
x0: 2.2780149131662516e-05


[['x0', 2.2780149131662516e-05]]

Plot the search progress:


In [6]:
#| label: fig-spot-progress
#| fig-cap: 'Visualization of the search progress of the `Spot` method. The black elements (points and line) represent the initial design, before the surrogate is build. The red elements represent the search on the surrogate.'
spot_0.plot_progress(log_y=True)

<Figure size 2700x1800 with 1 Axes>

In [7]:
#| label: fig-spot-model-1d
#| fig-cap: Visualization of the model and the underlying objective function values.
spot_0.plot_model()

<Figure size 1650x1050 with 1 Axes>

## `Spot` Parameters: `fun_evals`, `init_size` and `show_models`

We will modify three parameters:

  1. The number of function evaluations (`fun_evals`) will be set to `10` (instead of 15, which is the default value) in the `fun_control` dictionary.
  2. The parameter `show_models`, which visualizes the search process for each single iteration for 1-dim functions, in the `fun_control` dictionary.
  3. The size of the initial design (`init_size`) in the `design_control` dictionary.


The full list of the `Spot` parameters is shown in code reference on GitHub, see [Spot](https://sequential-parameter-optimization.github.io/spotPython/reference/spotPython/spot/spot/#spotPython.spot.spot).


In [8]:
fun_control=fun_control_init(lower = np.array([-1]),
                     upper = np.array([1]),
                     fun_evals = 10,
                     show_models = True)               
design_control = design_control_init(init_size=9)
spot_1 = spot.Spot(fun=fun,
                   fun_control=fun_control,
                   design_control=design_control)
spot_1.run()

Seed set to 123


<Figure size 1650x1050 with 1 Axes>

<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 1.1624309649016718e-08 [##########] 100.00% Done...



## Print the Results


In [9]:
spot_1.print_results()

min y: 1.1624309649016718e-08
x0: -0.00010781609179068177


[['x0', -0.00010781609179068177]]

## Show the Progress


In [10]:
spot_1.plot_progress()

<Figure size 2700x1800 with 1 Axes>

## Visualizing the Optimization and Hyperparameter Tuning Process with TensorBoard {#sec-visualizing-tensorboard-01}

`spotPython` supports the visualization of the hyperparameter tuning process with TensorBoard. The following example shows how to use TensorBoard with `spotPython`.

First, we define an "PREFIX" to identify the hyperparameter tuning process. The PREFIX is used to create a directory for the TensorBoard files.


In [11]:
fun_control = fun_control_init(
    PREFIX = "01",
    lower = np.array([-1]),
    upper = np.array([2]))
design_control = design_control_init(init_size=5)

Seed set to 123


Created spot_tensorboard_path: runs/spot_logs/01_p040025_2024-01-09_20-11-31 for SummaryWriter()


Since the `spot_tensorboard_path` is defined, `spotPython` will log the optimization process in the TensorBoard files. 


In [12]:
spot_tuner = spot.Spot(fun=fun,
                   fun_control=fun_control,
                   design_control=design_control)
spot_tuner.run()
spot_tuner.print_results()

spotPython tuning: 2.7704798574631272e-05 [####------] 40.00% 


spotPython tuning: 7.761255945313609e-07 [#####-----] 46.67% 


spotPython tuning: 7.729540372746933e-07 [#####-----] 53.33% 


spotPython tuning: 3.677408011584826e-07 [######----] 60.00% 


spotPython tuning: 1.1649997893485877e-09 [#######---] 66.67% 


...

spotPython tuning: 1.1649997893485877e-09 [#########-] 86.67% 


spotPython tuning: 1.1649997893485877e-09 [#########-] 93.33% 


spotPython tuning: 1.1649997893485877e-09 [##########] 100.00% Done...



min y: 1.1649997893485877e-09
x0: -3.4132093245926007e-05


[['x0', -3.4132093245926007e-05]]

Now we can start TensorBoard in the background. The TensorBoard process will read the TensorBoard files and visualize the hyperparameter tuning process.
From the terminal, we can start TensorBoard with the following command:



```{raw}
tensorboard --logdir="./runs"
```



`logdir` is the directory where the TensorBoard files are stored. In our case, the TensorBoard files are stored in the directory `./runs`.

TensorBoard will start a web server on port 6006. We can access the TensorBoard web server with the following URL:



```{raw}
http://localhost:6006/
```



The first TensorBoard visualization shows the objective function values plotted against the wall time. The wall time is the time that has passed since the start of the hyperparameter tuning process. The five initial design points are shown in the upper left region of the plot. The line visualizes the optimization process.
![TensorBoard visualization of the spotPython process. Objective function values plotted against wall time.](figures_static/01_tensorboard_01.png)

The second TensorBoard visualization shows the input values, i.e., $x_0$, plotted against the wall time.
![TensorBoard visualization of the spotPython process.](figures_static/01_tensorboard_02.png)

The third TensorBoard plot illustrates how `spotPython` can be used as a microscope for the internal mechanisms of the surrogate-based optimization process. Here, one important parameter, the learning rate $\theta$ of the Kriging surrogate is plotted against the number of optimization steps.

![TensorBoard visualization of the spotPython process.](figures_static/01_tensorboard_03.png){width="50%"}


## Jupyter Notebook

:::{.callout-note}

* The Jupyter-Notebook of this lecture is available on GitHub in the [Hyperparameter-Tuning-Cookbook Repository](https://github.com/sequential-parameter-optimization/Hyperparameter-Tuning-Cookbook/blob/main/007_num_spot_intro.ipynb)

:::